In [1]:
import pandas as pd

pd.__version__

'1.5.2'

In [2]:
# 示例数据框
demo_left = pd.DataFrame(
    {
        'left_id': ['a', 'b', 'c'],
        'datetime': ['2023-01-01', '2023-02-01', '2023-03-01']
    }
)
demo_right = pd.DataFrame(
    {
        'right_id': ['a', 'a', 'b', 'b'],
        'datetime': ['2022-12-28', '2023-01-06', '2023-02-15', '2023-02-02']
    }
)
demo_left['datetime'], demo_right['datetime'] = pd.to_datetime(demo_left['datetime']), pd.to_datetime(demo_right['datetime'])

In [3]:
demo_left

,left_id,datetime
0,a,2023-01-01
1,b,2023-02-01
2,c,2023-03-01


In [4]:
demo_right

,right_id,datetime
0,a,2022-12-28
1,a,2023-01-06
2,b,2023-02-15
3,b,2023-02-02


- 通常做法

In [5]:
(
    demo_left
    .merge(
        demo_right,
        left_on='left_id',
        right_on='right_id',
        suffixes=['_left', '_right']
    )
    .groupby('left_id', group_keys=False)
    .apply(
        lambda df: df.loc[(df.datetime_left - df.datetime_right).dt.days.abs() <= 7, :]
    )
)

,left_id,datetime_left,right_id,datetime_right
0,a,2023-01-01,a,2022-12-28
1,a,2023-01-01,a,2023-01-06
3,b,2023-02-01,b,2023-02-02


- 特殊做法

In [6]:
# pip install pyjanitor
import janitor

(
    demo_left
    # 添加辅助范围判断用上下限字段
    .assign(
        datetime_past_limit=demo_left['datetime'] - pd.Timedelta(days=7),
        datetime_future_limit=demo_left['datetime'] + pd.Timedelta(days=7)
    )
    # 执行条件连接
    .conditional_join(
        demo_right,
        ('left_id', 'right_id', '=='),
        ('datetime_past_limit', 'datetime', '<='),
        ('datetime_future_limit', 'datetime', '>=')
    )
)

left                                                         right  \
  left_id   datetime datetime_past_limit datetime_future_limit right_id   
0       a 2023-01-01          2022-12-25            2023-01-08        a   
1       a 2023-01-01          2022-12-25            2023-01-08        a   
2       b 2023-02-01          2023-01-25            2023-02-08        b   

              
    datetime  
0 2022-12-28  
1 2023-01-06  
2 2023-02-02